In [ ]:
# Importar configuraciones

import sys
import os
sys.path.append(os.path.abspath('./src'))

from config.spark_config import SparkSession
from etl.transformaciones import TransformacionesVentas
from pyspark.sql import functions as F
from config.hdfs_uploader import subir_a_hdfs

# Crear la sesión Spark con soporte Hive

In [ ]:
spark = SparkSession.getActiveSession()
if spark:
    spark.stop()

# Inicializar Spark
spark = SparkSession.builder \
    .appName("ExportData-Gold-To-CSV") \
    .enableHiveSupport() \
    .getOrCreate()

# Leer la tabla desde Hive Metastore

In [ ]:
df = spark.table("gold.transaccion_enriquecida")

In [ ]:
df = spark.sql("SELECT * FROM gold.transaccion_enriquecida")

# Guardar como CSV

In [ ]:
df.write \
  .mode("overwrite") \
  .option("header", "true") \
  .csv("data_hive/transaccion_enriquecida")


# UN SOLO archivo CSV

In [ ]:
df.coalesce(1).write \
  .mode("overwrite") \
  .option("header", "true") \
  .csv("dataset/transaccion_enriquecida_temp")

# Mover el archivo part-0000...csv
Renombrarlo a: `dataset/transaccion_enriquecida.csv`

In [ ]:
import os
import shutil

folder = "dataset/transaccion_enriquecida_temp"

for file in os.listdir(folder):
    if file.startswith("part-") and file.endswith(".csv"):
        shutil.move(os.path.join(folder, file),
                    "dataset/transaccion_enriquecida.csv")

shutil.rmtree(folder)